In [ ]:
from tests.test_attention import flash_backward_results, _attention_and_lse, _make_attn_inputs
from cs336_systems.flashattention_autograd_function import (
    FlashAttentionAutogradFunctionTriton, 
    FlashAttentionAutogradFunctionPytorch, 
    AttentionAutogradFunctionPytorch,     
)

# impl_0 = AttentionAutogradFunctionPytorch.apply
# q, k, v, do = _make_attn_inputs(device='cuda')
# o_0 = impl_0(q, k, v, True)
# o_0.backward(do)
# v.grad

In [ ]:
class FlashAttention2:
    def __init__(self, impl, B_q=16, B_k=16):
        self.impl = impl
        self.B_q = B_q
        self.B_k = B_k
    
    def __call__(self, Q, K, V, is_causal=True):
        return self.impl.apply(Q, K, V, is_causal, self.B_q, self.B_k)

# # Usage:
# flash_small = FlashAttentionPytorch(B_q=8, B_k=8)
# o = flash_small(q, k, v, False)

In [ ]:
import torch
import triton
def test_timing_flash_forward_backward(test, impl, n_heads, d_head, sequence_length, dtype=torch.bfloat16, device='cuda', B_q=16, B_k=16):
    q, k, v = torch.randn(
        3, n_heads, sequence_length, d_head, device=device, dtype=dtype, requires_grad=True
    )
    
    flash = torch.compile(FlashAttention2(impl, B_q, B_k))
    # flash = FlashAttention2(impl, B_q, B_k)
    
    def flash_forward():
        o = flash(q, k, v, True)

    def flash_forward_backward():
        o = flash(q, k, v, True)
        loss = o.sum()
        loss.backward()

    if test == "forward":
        results = triton.testing.do_bench(flash_forward, rep=1000, warmup=1000)
    elif test == "forward_backward":
        results = triton.testing.do_bench(flash_forward_backward, rep=1000, warmup=1000)
    else:
        raise ValueError("Wrong selection.")
    print(results)

In [ ]:
test_timing_flash_forward_backward("forward_backward", AttentionAutogradFunctionPytorch, 16, 64, 16384)

In [ ]:
test_timing_flash_forward_backward("forward_backward", FlashAttentionAutogradFunctionTriton, 16, 64, 16384, dtype=torch.bfloat16, B_q=32, B_k=32)